In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


#### Here is where we produce the model for digits 0-9


name = 'model_digits_1'                                 # CHANGE NAME OF MODEL HERE

batch_size = 64                                         # CHANGE BATCH SIZE HERE


print(tf.__version__)



############################################
####   Model (modified from jrobchin)   ####
#### 3 conv + pool layers + 2 NN layers ####
############################################


def createModel():                                                            # Create model function
    model = Sequential()                                             

    model.add(Conv2D(32, (3, 3), input_shape=(6, 10, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(batch_size))                       # 1 NN layer, FIRST LAYER must equal to batch size      
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4))                                # 1 NN layer, LAST LAYER must equal to number of classes
    model.add(Activation('softmax'))

    opt = tf.keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model


model1 = createModel()                                                      # Create the actual model



#####################################
#### Train model (from jrobchin) ####
#####################################

training_datagen = ImageDataGenerator(
    rotation_range=20,                                                     # original rotation_range = 50
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    zoom_range=0.2, rotation_range=10
)

training_generator = training_datagen.flow_from_directory(
    'hand_data/training',
    target_size=(54, 54),
    batch_size=batch_size,
    color_mode='grayscale'
)

validation_generator = validation_datagen.flow_from_directory(
    'hand_data/validation',
    target_size=(54, 54),
    batch_size=batch_size,
    color_mode='grayscale'
)

history = model1.fit_generator(
    generator=training_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=200 // batch_size,
    workers=8,
)


filename_model = name + ".hdf5"
model1.save(filename_model)











